In [14]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive
Mounted at /content/gdrive


In [2]:
%cd gdrive/MyDrive/IC_Leticia/Dataset

/content/gdrive/.shortcut-targets-by-id/16f9nXoTeh8KK8EltZTEP4Q38e0gcr6ne/IC_Leticia/Dataset


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-80d707dda84c>", line 1, in <cell line: 1>
    get_ipython().run_line_magic('cd', 'gdrive/MyDrive/IC_Leticia/Dataset')
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/in

In [15]:
!ls

Aquisição  GoEmotions  GoEmotions_pred.csv  seed_tweets.csv
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
ls: cannot open directory '.': Transport endpoint is not connected


In [16]:
!pip install -r "GoEmotions/requirements.txt"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.9/

In [5]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from pprint import pprint
import pandas as pd

#Folder path containing the fine-tuned model files
model_path = 'GoEmotions/fine_tuned_model'

model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, top_k=None)

threshold = 0.3

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-14697e587a3b>", line 8, in <cell line: 8>
    model = BertForSequenceClassification.from_pretrained(model_path)
  File "/usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py", line 2089, in from_pretrained
    use_safetensors = kwargs.pop("use_safetensors", None if is_safetensors_available() else False)
  File "/usr/local/lib/python3.9/dist-packages/transformers/utils/import_utils.py", line 587, in is_safetensors_available
    return importlib.util.find_spec("safetensors") is not None
  File "/usr/lib/python3.9/importlib/util.py", line 103, in find_spec
    return _find_spec(fullname, parent_path)
  File "<frozen importlib._bootstrap>", line 925, in _find_spec
  File "<frozen importlib._bootstrap_external>", line 1423, in find_spec
  File "<frozen import

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-14697e587a3b>", line 8, in <cell line: 8>
    model = BertForSequenceClassification.from_pretrained(model_path)
  File "/usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py", line 2089, in from_pretrained
    use_safetensors = kwargs.pop("use_safetensors", None if is_safetensors_available() else False)
  File "/usr/local/lib/python3.9/dist-packages/transformers/utils/import_utils.py", line 587, in is_safetensors_available
    return importlib.util.find_spec("safetensors") is not None
  File "/usr/lib/python3.9/importlib/util.py", line 103, in find_spec
    return _find_spec(fullname, parent_path)
  File "<frozen importlib._bootstrap>", line 925, in _find_spec
  File "<frozen importlib._bootstrap_external>", line 1423, in find_spec
  File "<frozen import

In [6]:
df = pd.read_csv("seed_tweets.csv",index_col=0)
inputs = df["seedtweet"].values.tolist()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-e0aa95de2929>", line 1, in <cell line: 1>
    df = pd.read_csv("seed_tweets.csv",index_col=0)
  File "/usr/local/lib/python3.9/dist-packages/pandas/util/_decorators.py", line 311, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/parsers/readers.py", line 678, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/parsers/readers.py", line 575, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/parsers/readers.py", line 932, in __init__
    self._engine = self._make_engine(f, self.engine)
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/parsers/readers.py", line 1216, in _make_engine
  

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-e0aa95de2929>", line 1, in <cell line: 1>
    df = pd.read_csv("seed_tweets.csv",index_col=0)
  File "/usr/local/lib/python3.9/dist-packages/pandas/util/_decorators.py", line 311, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/parsers/readers.py", line 678, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/parsers/readers.py", line 575, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/parsers/readers.py", line 932, in __init__
    self._engine = self._make_engine(f, self.engine)
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/parsers/readers.py", line 1216, in _make_engine
  

In [17]:
df.head()

,seedtweet
index,
0,"O Presidente @ArthurLira_ ,após ouvir a maior..."
1,"Dentre outros fatores, amanhã votarei pela li..."
2,STF decide manter a prisão do Deputado @danie...
3,O Conselho de Ética da Câmara vai começar a t...
4,Viúvas da ditadura se arvoram contra o STF ap...


,seedtweet,emotion,score
index,,,
0,"O Presidente @ArthurLira_ ,após ouvir a maior...",aprovação,0.558750
1,"Dentre outros fatores, amanhã votarei pela li...",neutro,0.618682
2,STF decide manter a prisão do Deputado @danie...,neutro,0.949404
3,O Conselho de Ética da Câmara vai começar a t...,curiosidade,0.377507
4,Viúvas da ditadura se arvoram contra o STF ap...,neutro,0.360773


In [18]:
output = classifier(inputs)
predictions = []

for prediction in output:
	predictions.append(list(x for x in prediction if x['score']>= threshold))

In [19]:
pprint(predictions)

[[{'label': 'aprovação', 'score': 0.5587501525878906}],
 [{'label': 'neutro', 'score': 0.6186824440956116}],
 [{'label': 'neutro', 'score': 0.9494044184684753}],
 [{'label': 'curiosidade', 'score': 0.3775072693824768}],
 [{'label': 'neutro', 'score': 0.36077290773391724}],
 [{'label': 'curiosidade', 'score': 0.5730711817741394}],
 [{'label': 'neutro', 'score': 0.9068360328674316}],
 [{'label': 'neutro', 'score': 0.6243780851364136}],
 [{'label': 'constrangimento', 'score': 0.5402638912200928}],
 [{'label': 'neutro', 'score': 0.6096152067184448}],
 [{'label': 'curiosidade', 'score': 0.5807187557220459}],
 [{'label': 'neutro', 'score': 0.6631458401679993}],
 [{'label': 'neutro', 'score': 0.8577451109886169}],
 [{'label': 'neutro', 'score': 0.9119518995285034}],
 [{'label': 'decepção', 'score': 0.46035119891166687}],
 [{'label': 'neutro', 'score': 0.7890118360519409}],
 [{'label': 'admiração', 'score': 0.37314921617507935}],
 [{'label': 'neutro', 'score': 0.6008589267730713}],
 [{'label':

In [20]:
emotions = []
scores = []
for row in predictions:
  if len(row)>0:
    first_result = row[0]
    print("Label: ",first_result.get("label")," - Scores:",first_result.get("score"))
    emotions.append(first_result.get("label"))
    scores.append(first_result.get("score"))
  else:
    print("Nan")
    emotions.append("NaN")
    scores.append(0)
  

Label:  aprovação  - Scores: 0.5587501525878906
Label:  neutro  - Scores: 0.6186824440956116
Label:  neutro  - Scores: 0.9494044184684753
Label:  curiosidade  - Scores: 0.3775072693824768
Label:  neutro  - Scores: 0.36077290773391724
Label:  curiosidade  - Scores: 0.5730711817741394
Label:  neutro  - Scores: 0.9068360328674316
Label:  neutro  - Scores: 0.6243780851364136
Label:  constrangimento  - Scores: 0.5402638912200928
Label:  neutro  - Scores: 0.6096152067184448
Label:  curiosidade  - Scores: 0.5807187557220459
Label:  neutro  - Scores: 0.6631458401679993
Label:  neutro  - Scores: 0.8577451109886169
Label:  neutro  - Scores: 0.9119518995285034
Label:  decepção  - Scores: 0.46035119891166687
Label:  neutro  - Scores: 0.7890118360519409
Label:  admiração  - Scores: 0.37314921617507935
Label:  neutro  - Scores: 0.6008589267730713
Label:  admiração  - Scores: 0.3365543484687805
Nan
Label:  desaprovação  - Scores: 0.6816696524620056
Label:  otimismo  - Scores: 0.4443388283252716
Nan
L

In [21]:
df["emotion"] = emotions
df["score"] = scores

In [22]:
df.head()

,seedtweet,emotion,score
index,,,
0,"O Presidente @ArthurLira_ ,após ouvir a maior...",aprovação,0.558750
1,"Dentre outros fatores, amanhã votarei pela li...",neutro,0.618682
2,STF decide manter a prisão do Deputado @danie...,neutro,0.949404
3,O Conselho de Ética da Câmara vai começar a t...,curiosidade,0.377507
4,Viúvas da ditadura se arvoram contra o STF ap...,neutro,0.360773


,seedtweet,emotion,score
index,,,
0,"O Presidente @ArthurLira_ ,após ouvir a maior...",aprovação,0.558750
1,"Dentre outros fatores, amanhã votarei pela li...",neutro,0.618682
2,STF decide manter a prisão do Deputado @danie...,neutro,0.949404
3,O Conselho de Ética da Câmara vai começar a t...,curiosidade,0.377507
4,Viúvas da ditadura se arvoram contra o STF ap...,neutro,0.360773


In [13]:
#df = pd.DataFrame(df, reset_index=True, columns=["predicoes"])
df.to_csv("GoEmotions_pred.csv")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-23-832868f7c2c2>", line 2, in <cell line: 2>
    df.to_csv("GoEmotions_pred.csv")
  File "/usr/local/lib/python3.9/dist-packages/pandas/core/generic.py", line 3551, in to_csv
    return DataFrameRenderer(formatter).to_csv(
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/formats/format.py", line 1180, in to_csv
    csv_formatter.save()
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/formats/csvs.py", line 241, in save
    with get_handle(
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/common.py", line 694, in get_handle
    check_parent_directory(str(handle))
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/common.py", line 567, in check_parent_directory
    if not parent.is_dir():
  File "/usr/lib/python3.9/pathlib.py", line 1439, in is_

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-23-832868f7c2c2>", line 2, in <cell line: 2>
    df.to_csv("GoEmotions_pred.csv")
  File "/usr/local/lib/python3.9/dist-packages/pandas/core/generic.py", line 3551, in to_csv
    return DataFrameRenderer(formatter).to_csv(
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/formats/format.py", line 1180, in to_csv
    csv_formatter.save()
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/formats/csvs.py", line 241, in save
    with get_handle(
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/common.py", line 694, in get_handle
    check_parent_directory(str(handle))
  File "/usr/local/lib/python3.9/dist-packages/pandas/io/common.py", line 567, in check_parent_directory
    if not parent.is_dir():
  File "/usr/lib/python3.9/pathlib.py", line 1439, in is_